In [4]:
import OpenDartReader
import pandas as pd
from datetime import datetime
import time
import warnings
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser

warnings.simplefilter("ignore")
api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘

#code : 종목코드
#corp : 종목이름
#fs_div : 분기 1(11013),2(11012),3(11014),4(11011)
#reprt_code : 연결, 개별 ( OFC, CFS)

def quater_fin(code,corp,year,reprt_code,fs_div):
    # try:
    df = dart.finstate_all(code, year,reprt_code=reprt_code,fs_div=fs_div)
    df = df[df['sj_nm'].str.contains('손익')]
    print(df)
    df = df[df['account_id'].str.contains('ifrs-full_Revenue')|df['account_id'].str.contains('OperatingIncomeLoss')]
    df = df[['account_nm','thstrm_amount']]

    if reprt_code == '11014':
        df.columns = ['항목',f'{year}.3Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))
        
        # sales = df.loc[(corp, '매출액'), f'{year}.3Q']
        # op = df.loc[(corp, '영업이익'), f'{year}.3Q']
        # opm = (op / sales) * 100  # 백분율로 변환
        # df.loc[(corp, 'OPM'), f'{year}.3Q'] = opm
        # df = df.apply(lambda x : round(x,1))  
        
        
    elif reprt_code == '11012':
        df.columns = ['항목',f'{year}.2Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))   
        
        # sales = df.loc[(corp, '매출액'), f'{year}.2Q']
        # op = df.loc[(corp, '영업이익'), f'{year}.2Q']
        # opm = (op / sales) * 100  # 백분율로 변환
        # df.loc[(corp, 'OPM'), f'{year}.2Q'] = opm
        # df = df.apply(lambda x : round(x,1))  
         
         
    elif reprt_code == '11013':
        df.columns = ['항목',f'{year}.1Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))  
        
        # sales = df.loc[(corp, '매출액'), f'{year}.1Q']
        # op = df.loc[(corp, '영업이익'), f'{year}.1Q']
        # opm = (op / sales) * 100  # 백분율로 변환
        # df.loc[(corp, 'OPM'), f'{year}.1Q'] = opm
        # df = df.apply(lambda x : round(x,1))  
       
       
    elif reprt_code == '11011':
        df.columns = ['항목',f'{year}.4Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))    

        # sales = df.loc[(corp, '매출액'), f'{year}.4Q']
        # op = df.loc[(corp, '영업이익'), f'{year}.4Q']
        # opm = (op / sales) * 100  # 백분율로 변환
        # df.loc[(corp, 'OPM'), f'{year}.4Q'] = opm
        # df = df.apply(lambda x : round(x,1))  

    # if fs_div == 'CFS':
    #     df.insert(0,'구분','연결')
        
    # else: 
    #     df.insert(0,'구분','별도')
    # except Exception as e:
    #     print(corp,e)
    time.sleep(2)
    return df

#4분기 조정 ( 연간에서 분기로)
def quarter_4_adj(corp, df):
    print(corp, df)
    try:
        # 각 연도별로 4분기 조정을 위해 컬럼이 존재하는지 확인하고 처리
        if '2023.4Q' in df.columns:
            df['2023.4Q'] = df['2023.4Q'] - (df.get('2023.3Q', 0) + df.get('2023.2Q', 0) + df.get('2023.1Q', 0))
        
        if '2022.4Q' in df.columns:
            df['2022.4Q'] = df['2022.4Q'] - (df.get('2022.3Q', 0) + df.get('2022.2Q', 0) + df.get('2022.1Q', 0))
        
        if '2021.4Q' in df.columns:
            df['2021.4Q'] = df['2021.4Q'] - (df.get('2021.3Q', 0) + df.get('2021.2Q', 0) + df.get('2021.1Q', 0))
        
        if '2020.4Q' in df.columns:
            df['2020.4Q'] = df['2020.4Q'] - (df.get('2020.3Q', 0) + df.get('2020.2Q', 0) + df.get('2020.1Q', 0))

        # 데이터프레임 전치
        df = df.T

        df[corp,'opm'] = round(df[corp,'영업이익']/df[corp,'매출액']*100,1)
        df[corp,'yoy'] = round((df[corp,'매출액'] - df[corp,'매출액'].shift(4))/df[corp,'매출액'].shift(4)*100,1)
        print(df)
        # 다시 전치
        df = df.T

    except Exception as e:
        print(f"Error: {e}")
    return df
    
# 직원수 확인

def employee_cnt(code,corp,year):

    lists = dart.list(code, start='2020-04-01',end = '2020-06-30', kind='A') 
    lists['공시'] = lists['report_nm'].str.split(' ').str[1]

    rcp_no = []
    for i in range(0,len(lists)):
        rcp = lists['rcept_no'][i]
        rcp_no.append(rcp)  
    print(rcp_no)

    df = pd.DataFrame()
    
    for i in range(0,len(rcp_no)):
        print(i)
    #     try:    
        files = dart.sub_docs(rcp_no[i])
        content = [] #

        data = files[files['title'].str.contains('1. 임원')]
        
        url = data['url'].values[0]
        
        res = requests.get(url)
        soup=BeautifulSoup(res.text,'html.parser')
        temps = soup.select('body > table')
        
                    
        for temp in temps:
    #             # print(v)
            text = temp.get_text()
            
            if '직원' in text or '직 원' in text : 
    #                 # print(text)
                content.append(temp)
        
        if '급여' in content[0].text.strip():
            table = parser.make2d(content[0]) #표로 만들어줌
        elif '급여' in content[1].text.strip():
            table = parser.make2d(content[1]) #표로 만들어줌
        elif '급여' in content[2].text.strip():
            table = parser.make2d(content[2]) #표로 만들어줌
        elif '급여' in content[3].text.strip():
            table = parser.make2d(content[3]) #표로 만들어줌
        emp_table = pd.DataFrame(table) # 필요한부분만 불러옴
 
        emp_table.columns = emp_table.iloc[2]
        emp_table = emp_table.iloc[-1:,:]
        emp_table = emp_table[['합 계']]

            
        emp_table.insert(0,'날짜',lists['공시'][i])
        emp_table['날짜'] = emp_table['날짜'].str.replace('(','',regex=True)
        emp_table['날짜'] = emp_table['날짜'].str.replace(')','',regex=True)
        print(emp_table)
        emp_table.insert(0,'기업',corp)
        print(emp_table)
                
        df = pd.concat([df,emp_table],ignore_index=True)
        print(df)
    #         time.sleep(1)
            
    #     except Exception as e:
    #         print(f'{corp_name}',e)
    #     pass
        return df

# code = '302550'
# corp = '리메드'
# year = '2020'
# temp = employee_cnt(code,corp,year)
# temp
#=====================================

#fs_div : 분기 1(11013),2(11012),3(11014),4(11011)
code = '054210'
corp = '이랜텍'
# years = ['2020','2021','2022','2023','2024']
years = ['2019'] # 평가

reprt_codes = ['11013','11012','11014','11011']
# reprt_codes = ['11013'] # 평가


result_df = pd.DataFrame()
for year in years:
    for reprt_code in reprt_codes:
        try:
            fs_div = 'CFS' # OFS 개별, CFS 연결
            df = quater_fin(code,corp,year,reprt_code,fs_div)
            result_df = pd.concat([result_df,df],axis=1)
            
        except Exception as e:
            try:
                fs_div = 'OFS'  # OFS 개별
                df = quater_fin(code, corp, year, reprt_code, fs_div)
                print(df)
                result_df = pd.concat([result_df,df],axis=1)
                
            except Exception as e:
                print(e)
ttl = quarter_4_adj(corp,result_df)

ttl
# ttl.to_clipboard()
# ttl.to_excel(f'./개별종목_재무/{corp}_{today}.xlsx')


reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
          rcept_no reprt_code bsns_year corp_code sj_div    sj_nm  \
54  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
55  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
56  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
57  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
58  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
59  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
60  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
61  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
62  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
63  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
64  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
65  20190515000896      11013      2019  00206659    CIS  포괄손익계산서   
66  20190515000896      11013      2019  00206659   

2019.3Q  2019.4Q
기업  항목                    
이랜텍 매출액    2167.0   7290.0
    영업이익    143.0    363.0
    opm       6.6      5.0
    yoy       NaN      NaN